下面制作一个自定义model，以为自定义CNN做基础。

In [5]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

def my_model(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)
    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.0001)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

class DataSetLoader:
    def __init__(self, data_dir):
        # Set FLAGS.charset_size to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

    
train_data = DataSetLoader(data_dir='../data/train_/')
test_data = DataSetLoader(data_dir='../data/test_/')
print ('Data loaded ...')


# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build 2 layer DNN classifier
'''
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=CHARSET_SIZE,
    dropout=0.1,
    model_dir="../dfs/checkpoint/customized_model-a"
)
'''

# Build 2 hidden layer DNN with ...
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns': feature_columns,
        # Two hidden layers of ...
        'hidden_units': [512, 512,512,512,512],
        # The model must choose between CHARSET_SIZE classes.
        'n_classes': CHARSET_SIZE,
    },
     model_dir='../dfs/checkpoint/customized_model-b')

    

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(train_data)[0]},
    y=input(train_data)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)
print ('Begin to train ...')

classifier.train(input_fn=train_input_fn, steps=10000)
print ('Train done ...')

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

Now processing path:  ../data/train_/00037
Now processing path:  ../data/test_/00037
Data loaded ...
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/customized_model-b', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Begin to train ...
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-b/model.ckpt-20000
INFO:tensorflow:Saving checkpoints for 20001 into ../dfs/checkpoint/customized_model-b/model.ckpt.
INFO:tensorflow:loss = 2.24881, step = 20001
INFO:tensorflow:global_step/sec: 30.6042
INFO:tensorflow:loss = 2.02769, step = 20101 (3.269 sec)
INFO:tensorflow:global_step/sec: 31.0383
INFO:tensorflow:loss = 1.89215, step = 20201 (3.223 sec)
INFO:tensorflow:global_step/sec: 31.3634

INFO:tensorflow:global_step/sec: 31.0725
INFO:tensorflow:loss = 1.67826, step = 27701 (3.218 sec)
INFO:tensorflow:global_step/sec: 30.7437
INFO:tensorflow:loss = 1.69541, step = 27801 (3.253 sec)
INFO:tensorflow:global_step/sec: 31.0679
INFO:tensorflow:loss = 1.79343, step = 27901 (3.219 sec)
INFO:tensorflow:global_step/sec: 31.6557
INFO:tensorflow:loss = 1.82311, step = 28001 (3.159 sec)
INFO:tensorflow:global_step/sec: 31.1543
INFO:tensorflow:loss = 1.83987, step = 28101 (3.210 sec)
INFO:tensorflow:global_step/sec: 31.0804
INFO:tensorflow:loss = 1.6753, step = 28201 (3.218 sec)
INFO:tensorflow:global_step/sec: 31.1077
INFO:tensorflow:loss = 1.65858, step = 28301 (3.215 sec)
INFO:tensorflow:global_step/sec: 30.9249
INFO:tensorflow:loss = 1.642, step = 28401 (3.233 sec)
INFO:tensorflow:global_step/sec: 31.2943
INFO:tensorflow:loss = 1.80391, step = 28501 (3.195 sec)
INFO:tensorflow:global_step/sec: 31.5225
INFO:tensorflow:loss = 1.65393, step = 28601 (3.172 sec)
INFO:tensorflow:global_

自定义模型：

INFO:tensorflow:Starting evaluation at 2018-04-25-01:50:24
INFO:tensorflow:Restoring parameters from /tmp/tmp8r0rmwe3/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2018-04-25-01:50:24
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.276471, global_step = 2000, loss = 2.90342

Test Accuracy: 27.647060%

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.0001)

上面是learning rate为千分之一时，执行2000步的结果；

下面我们试试10^-4时，4000步时的结果：

INFO:tensorflow:Starting evaluation at 2018-04-25-02:29:34
INFO:tensorflow:Restoring parameters from /tmp/tmptu8djitd/model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2018-04-25-02:29:35
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.360633, global_step = 10000, loss = 2.9688

Test Accuracy: 36.063349%


使用5层：

INFO:tensorflow:Starting evaluation at 2018-04-25-02:54:05
INFO:tensorflow:Restoring parameters from /tmp/tmpg4mwuwyt/model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-04-25-02:54:05
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.49457, global_step = 20000, loss = 2.10177

Test Accuracy: 49.457014%


INFO:tensorflow:Starting evaluation at 2018-04-25-03:45:11
INFO:tensorflow:Restoring parameters from /tmp/tmpzysjsz1b/model.ckpt-50000
INFO:tensorflow:Finished evaluation at 2018-04-25-03:45:12
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.570136, global_step = 50000, loss = 1.54011

Test Accuracy: 57.013577%
这个是有长进的。


====
重新加了model dir，进行训练：

INFO:tensorflow:Starting evaluation at 2018-04-26-04:31:35
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-b/model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2018-04-26-04:31:35
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.365158, global_step = 10000, loss = 2.96748

Test Accuracy: 36.515838%

第二次训练，果真继续上次的checkpoint:
INFO:tensorflow:Starting evaluation at 2018-04-26-04:38:03
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-b/model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-04-26-04:38:03
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.502715, global_step = 20000, loss = 2.10188

Test Accuracy: 50.271493%


INFO:tensorflow:Starting evaluation at 2018-04-26-04:54:34
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-b/model.ckpt-30000
INFO:tensorflow:Finished evaluation at 2018-04-26-04:54:35
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.534842, global_step = 30000, loss = 1.78702

Test Accuracy: 53.484166%



